In [1]:
%reset -f

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


The block below identifies ghcnd stations with at least some data in the 1974-2013 range, record completeness > 0%, and distance to nearest CLIGEN station up to 100000 m. Only a metadata file is created, and the records are not downloaded in this step. A file is read containing lat/lon coordinates of the U.S. CLIGEN network. Run time is ~27 min.

In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime as dt
from datetime import timedelta
import os
import scipy as scipy
import statsmodels.api as sm
from google.colab import data_table
data_table.enable_dataframe_formatter()
import csv
import geopandas as gpd
from geopy.distance import geodesic


keepFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/Keepers_John.csv'
AnuragDIR = '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations'

folders = os.listdir(AnuragDIR)

folders = [f for f in folders if f[0].isdigit() and f != '24_us_stations.csv']

parPATHS = []
for folder in folders:
  files = os.listdir(os.path.join(AnuragDIR, folder))
  for f in files:
    if f[-4:] == '.par':
      parPATHS.append(os.path.join(AnuragDIR, folder, f))

print(parPATHS)
print(len(parPATHS))

df = pd.read_csv(keepFILE)

parNames = [os.path.split(p)[1][:-4] for p in parPATHS]
print(parNames)
df = df[df['stationID_left'].isin(parNames)]
print(df)
keepers = df['stationID_right'].values
print(keepers)


['/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/10_Illinois/il110510.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/11_Arkansas/ar032443.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/12_Texas/tx412121.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/13_Kansas/ks146549.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/14_Colorado/co050114.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/15_Montana/mt248169.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/16_Arizona/az024829.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/17_California/ca040449.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/18_Idaho/id104670.par', '/content/drive/My Drive/Colab Notebooks/TeamProjects/24_us_stations/19_Washington/wa454679.par', '/content/drive/My Drive/Colab Notebooks/Team

The block below downloads the identified records. Run time is ~6 hr.

In [ ]:
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import requests
import os

keepFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/Keepers_John.csv'
dataLINK = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access'
outFOLDER = '/content/drive/My Drive/Colab Notebooks/TeamProjects/Output/GHCN_Dataframes_for_Keepers_John'
keepers_df = pd.read_csv(keepFILE)
keepers = keepers_df['stationID_right'].values

dates = []
start_date = dt(1974, 1, 1, 0, 0)
end_date = dt(2013, 12, 31, 0, 0)
date = start_date
while date <= end_date:
  dates.append(date)
  date = date + timedelta(days=1)

i = 0
done = False
while done == False:

  keeper = keepers[i]
  print(keeper)
  df = pd.DataFrame()
  df.index.name = 'index'
  df['year'] = [date.year for date in dates]
  df['month'] = [date.month for date in dates]
  df['day'] = [date.day for date in dates]
  df[keeper] = [0.0 for date in dates]

  outFILE = os.path.join(outFOLDER, keeper + '.csv')
  url = dataLINK + '/' + keeper + '.csv'
  req = requests.get(url)
  text = req.text
  if not '404 Not Found' in text:

    lines = [line for line in text.splitlines()]
    hdrs = lines[0].split(',')
    prcp_col_i = hdrs.index('"PRCP"')
    date_col_i = hdrs.index('"DATE"')

    for line in lines[1:]:
      row = line.split('","')
      name_no_comma = row[5].replace(',', '')
      line = line.replace(row[5], name_no_comma)
      line = line.replace('","', ',')
      row = line.split(',')
      lon = row[3]
      lat = row[2]
      date = dt.strptime(row[date_col_i].strip('"'), '%Y-%m-%d')
      prcp = row[prcp_col_i].strip('"')

      if date.year >= 1974 and date.year <= 2013:

        if prcp != '' and not any([s in prcp for s in ['P', 'T', 'H', '9999']]):
          prcp = float(prcp)/10.0
          df.loc[(df['year'] == date.year) & (df['month']==date.month) & (df['day']==date.day), keeper] = prcp

  i += 1

  if i == len(keepers):
    done = True

  df.to_csv(outFILE)




In [ ]:
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import requests
import os


infoFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/station_info.csv'
keyFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/cligenGhcnd_Anurag_Key.txt'
info_df = pd.read_csv(infoFILE)
key_df = pd.read_csv(keyFILE, names=['cligen', 'ghcn'], sep='-')
key_df = pd.DataFrame(data=list(zip(key_df['cligen'].str.lstrip().str.rstrip().str.lower().values, key_df['ghcn'].str.lstrip().str.rstrip().values)), columns=['cligen', 'ghcn'])

print(key_df)

matches, keepers = [], []
for item in info_df['stationID_left']:
  print(item)

  l = key_df.loc[key_df['cligen'] == item]['ghcn'].reset_index(drop=True).get(0)
  r = info_df.loc[info_df['stationID_left'] == item]['stationID_right'].reset_index(drop=True).get(0)
  print(r)
  if l == r:
    print('hi')

  matches.append(l)
  keepers.append(l)


        cligen         ghcn
0     ak500352  USW00025308
1     ak500546  USW00027502
2     ak500657  USC00500657
3     ak500754  USW00026615
4     ak500761  USW00026533
...        ...          ...
2760  wy489025  USC00489025
2761  wy489205  USC00489205
2762  wy489615  USC00489615
2763  wy489770  USC00489770
2764  wy489905  USC00489905

[2765 rows x 2 columns]
nv264436
USW00023169
hi
tx412121
USC00412121
hi
or354811
USC00354811
hi
wi470516
USC00470516
hi
il110510
USW00013802
hi
id104670
USC00104670
hi
ar032443
USW00093993
hi
oh336405
USC00336405
hi
az028619
USC00028619
hi
ut425733
USC00425733
hi
ms221707
USC00221707
hi
ia133438
USC00133438
hi
fl081544
USC00081544
hi
ca040449
USC00040449
hi
wa454679
USC00454679
hi
nd328872
USC00328872
hi
mt248169
USC00248169
hi
az024829
US1AZMR0111
co050114
US1COWA0060
ny300448
US1NYST0029
ks146549
US1KSPR0024
nc310286
US1NCRN0023
ne255565
US10kear015
nm290234
USW00023004


In [ ]:
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import requests
import os

infoFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/New_GHCN_Station_Info.csv'
dataLINK = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access'
outFOLDER = '/content/drive/My Drive/Colab Notebooks/TeamProjects/Output/New_Anurag_Output'

dates = []
start_date = dt(1974, 1, 1, 0, 0)
end_date = dt(2013, 12, 31, 0, 0)
date = start_date
while date <= end_date:
  dates.append(date)
  date = date + timedelta(days=1)

i = 0
done = False
while done == False:

  keeper = keepers[i]
  print(keeper)
  df = pd.DataFrame()
  df.index.name = 'index'
  df['year'] = [date.year for date in dates]
  df['month'] = [date.month for date in dates]
  df['day'] = [date.day for date in dates]
  df[keeper] = [0.0 for date in dates]

  outFILE = os.path.join(outFOLDER, keeper + '.csv')
  url = dataLINK + '/' + keeper + '.csv'
  req = requests.get(url)
  text = req.text
  if not '404 Not Found' in text:

    lines = [line for line in text.splitlines()]
    hdrs = lines[0].split(',')
    prcp_col_i = hdrs.index('"PRCP"')
    date_col_i = hdrs.index('"DATE"')

    for line in lines[1:]:
      row = line.split('","')
      name_no_comma = row[5].replace(',', '')
      line = line.replace(row[5], name_no_comma)
      line = line.replace('","', ',')
      row = line.split(',')
      lon = row[3]
      lat = row[2]
      date = dt.strptime(row[date_col_i].strip('"'), '%Y-%m-%d')
      prcp = row[prcp_col_i].strip('"')

      if date.year >= 1974 and date.year <= 2013:

        if prcp != '' and not any([s in prcp for s in ['P', 'T', 'H', '9999']]):
          prcp = float(prcp)/10.0
          df.loc[(df['year'] == date.year) & (df['month']==date.month) & (df['day']==date.day), keeper] = prcp

  i += 1

  if i == len(keepers):
    done = True

  df.to_csv(outFILE)




USW00023169
USC00412121
USC00354811
USC00470516
USW00013802
USC00104670
USW00093993
USC00336405
USC00028619
USC00425733
USC00221707
USC00133438
USC00081544
USC00040449
USC00454679
USC00328872
USC00248169
USC00024829
USW00024015
USC00300448
USC00146549
USC00310286
USC00255565
USW00023050


In [ ]:
infoFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/station_info.csv'
newinfoFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/new_station_info.csv'
keyFILE = '/content/drive/My Drive/Colab Notebooks/TeamProjects/cligenGhcnd_Anurag_Key.txt'
info_df = pd.read_csv(infoFILE)
key_df = pd.read_csv(keyFILE, names=['cligen', 'ghcn'], sep='-')
key_df = pd.DataFrame(data=list(zip(key_df['cligen'].str.lstrip().str.rstrip().str.lower().values, key_df['ghcn'].str.lstrip().str.rstrip().values)), columns=['cligen', 'ghcn'])

print(key_df)
print(info_df)
cligen_stations = info_df['stationID_left'].values



keepers_info_df = pd.DataFrame(columns=['cligen', 'ghcn'])
keepers_info_df['cligen'] = cligen_stations


bad = []
for i, keeper in enumerate(keepers):
  keepers_info_df.at[i, 'ghcn'] = keeper

  try:
    ct = 0
    url = dataLINK + '/' + keeper + '.csv'
    req = requests.get(url)
    text = req.text
    if not '404 Not Found' in text:
      ct = 0
      lines = [line for line in text.splitlines()]
      save_lines = []
      hdrs = lines[0].split(',')
      prcp_i = hdrs.index('"PRCP"')
      date_i = hdrs.index('"DATE"')
      for line in lines[1:]:
        row = line.split('","')
        name_no_comma = row[5].replace(',', '')
        line = line.replace(row[5], name_no_comma)
        line = line.replace('","', ',')
        row = line.split(',')
        date = dt.strptime(row[date_i].strip('"'), '%Y-%m-%d')
        prcp = row[prcp_i].strip('"')
        if date.year >= 1974 and date.year <= 2013:
          if prcp != '' and not any([s in prcp for s in ['P', 'T', 'H', '9999']]):
            prcp = float(prcp)
            ct += 1

      error_per = float(ct)/float(len(dates))*100
      keepers_info_df.at[i, 'complete_per'] = error_per

    else:
      bad.append(keeper)

  except requests.exceptions.Timeout:
    bad.append(keeper)




keepers_info_df.to_csv(newinfoFILE)


        cligen         ghcn
0     ak500352  USW00025308
1     ak500546  USW00027502
2     ak500657  USC00500657
3     ak500754  USW00026615
4     ak500761  USW00026533
...        ...          ...
2760  wy489025  USC00489025
2761  wy489205  USC00489205
2762  wy489615  USC00489615
2763  wy489770  USC00489770
2764  wy489905  USC00489905

[2765 rows x 2 columns]
    index stationID_left  long_left  lat_left stationID_right  long_right  \
0       1       nv264436    -115.17     36.08     USW00023169   -115.1633   
1       2       tx412121    -101.22     33.65     USC00412121   -101.2778   
2       3       or354811    -122.68     44.10     USC00354811   -122.6886   
3       4       wi470516     -89.73     43.47     USC00470516    -89.7269   
4       5       il110510     -89.85     38.53     USW00013802    -89.8467   
5       6       id104670    -114.53     42.72     USC00104670   -114.5192   
6       7       ar032443     -94.17     36.00     USW00093993    -94.1683   
7       8       oh33640